Тут нарисован прототип типа-монте-карло симуляции работы инвестиционного фонда. Данные пока все выдуманы из головы, а не взяты с рынка. Я если честно не осилил в твоих мега excel-ях разобраться, предлагаю потом постепенно модель усложнять, если это то что тебе надо.

Основная идея такая у нас есть 10000 вселенных, в каждой вселенной 1 фонд и 10000 компаний.

Все вселенные за раз продвигаются на один цикл. Все рассчеты векторные, т.е. обсчитывают сразу 100M компаний во всех вселенных. Вся хуйня рассчитана допустим на 20 циклов, компании появляются не сразу, а равными долями типа по 1/20 в цикл.

На каждом цикле считается cash остаток фонда и стоимость доли компаний в его управлении

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
#тут описана модель. У нас сейчас 10000 вселенных, по 10000 компаний в каждой.
time_length = 20
universes   = 10000
companies   = 10000
capitalizations = np.zeros((universes,companies), dtype=float)  #это капитализация компаний
fund_share = np.zeros((universes,companies), dtype=float) #это доля фонда в каждой
fund_cash = np.ones((universes,), dtype=float)*6000000000.0 


тут отдельные процедурные шаги, пока пропусти читать иди читай в time_step

In [3]:
def create_new_companies(caps, step):
    
    fr = companies//time_length*step
    to = companies//time_length*(step+1)
    
    INIT_CAP=30000000.0 #средняя оценка которую получает компания на старте 
    CAP_STD=5000000.0 #67% укладываеются в среднюю +/- это число
    
    caps[:,fr:to]=np.random.normal(loc=INIT_CAP,scale=CAP_STD,size=(universes, to-fr))
    
    # у нас есть матрица компаний и вселеных, мы на каждом шаге заполняем новые компании
    # 0 0 0 0 0 0  =>  X X 0 0 0 0  =>  X X Y Y 0 0
    # 0 0 0 0 0 0  =>  X X 0 0 0 0  =>  X X Y Y 0 0
    

In [4]:
def fund_invest_new_companies(caps, share, cash, step):

    fr = companies//time_length*step
    to = companies//time_length*(step+1)

    INVEST_PROBABILITY=0.30
    lucky_ones = np.zeros((universes,companies), dtype=bool)
    # тут сейчас массив в котором 0 на тех местах где не инвестируем, и 1 где инвестируем.
    # 0 0
    # 0 1
    lucky_ones[:, fr:to] = (np.random.uniform(size=(universes, to-fr)) < INVEST_PROBABILITY)
    # тут мы его вставили в массив всех компаний не только новых
    # 0 0 0 0 0 0 
    # 0 0 0 1 0 0 
    
    INVEST_AMOUNT=0.07
    amounts = lucky_ones*INVEST_AMOUNT
    #тут сейчас у нас 7% там где хотим инвестировать.    
    # 0 0    
    # 0 0.07 

    share += amounts
    #добавили наше владение в shares, тут в этом случае вообще-то перед этим было у всех 0
    # 0 0    0 0      0 0
    # 0 0  + 0 0.07 = 0 0.07

    #надо понять сколько денег мы потратили
    money = caps * amounts
    # 30M 30M    0 0      0 0
    # 30M 30M  * 0 0.07 = 0 2.1M

    cash -= np.sum(money, axis=1)


In [5]:
def progress_old_companies(caps, share, cash, step):
    
    fr = companies//time_length*step
    
    if fr==0: return
    
    NOTHING_PROB=0.5
    NEXT_ROUND_PROB=0.1
    DEAD_ZOMBIE_PROB=0.2
    SOLD_PROB=0.05
    DIVIDENDS_PROB=0.15

    
    probs = np.array([NOTHING_PROB, NEXT_ROUND_PROB, DEAD_ZOMBIE_PROB,SOLD_PROB,DIVIDENDS_PROB])   
    results = np.zeros((universes,companies))
    results[:,0:fr] = np.random.choice(probs.shape[0], size=(universes,fr), replace=True, p=probs)    
    
    def do_nothing(mask, caps, share, cash, step):
        pass
    
    def dead_zombie(mask, caps, share, cash, step):
        caps[mask]=0
    
    def next_round(mask, caps, share, cash, step):
    
        n = np.sum(mask) #количество компаний в выборке.
        
        #  тут походу надо сделать по твоей табличке, но мне лень поэтому я 
        #  сделаю тупо вот так сам переделай как надо
        
        #  на каждой стадии валуация растет в среднем 10 раз, со стандартным отклонением в 3 раза.
        
        INCREASE=5
        INCREASE_STD=2
        
        val_increment = np.random.normal(loc=INCREASE,scale=INCREASE_STD, size=n)
        
        caps[mask]*=val_increment
        
    def sold(mask, caps, share, cash, step):

        # посчитали сколько стоит наша доля.
        money = caps * mask
        
        #обнулили нашу долю
        share[mask]=0
        
        #сложили по столбцам в каждой вселенной
        cash+=np.sum(money, axis=1)

    def dividends(mask, caps, share, cash, step):
        
        # посчитали сколько стоит наша доля.
        money = caps * mask

        # для каждой компании посчитали дивиденты в зависимости от капитализации
        
        DIV_TARGET=5
        DIV_STD=2
        
        dividends = np.random.normal(loc=DIV_TARGET,scale=DIV_STD, size=(universes, companies))
                
        #сложили по столбцам в каждой вселенной, добавили в cash фонда в этой вселенной
        cash+=np.sum(money, axis=1)

        
    results[caps==0]=0  #с мертвыми и зомби никогда ничего не происходит.
    do_nothing(results==0, caps, share, cash, step)
    next_round(results==1, caps, share, cash, step)
    dead_zombie(results==2, caps, share, cash, step)
    sold(results==3, caps, share, cash, step)
    dividends(results==4, caps, share, cash, step)



In [6]:
def fund_invest_old_companies(caps, share, cash, step):

    fr = companies//time_length*step
    to = companies//time_length*(step+1)

    INVEST_PROBABILITY=0.10
    lucky_ones = np.zeros((universes,companies), dtype=bool)
    lucky_ones[:, fr:to] = (np.random.uniform(size=(universes, to-fr)) < INVEST_PROBABILITY)
    
    INVEST_AMOUNT=0.1
    amounts = lucky_ones*INVEST_AMOUNT

    share += amounts

    #надо понять сколько денег мы потратили
    money = caps * amounts

    cash -= np.sum(money, axis=1)



In [7]:
time = 0 #мультивселенная создана

In [8]:
def time_step(caps, share, cash, step): #Это функция которая двигает время на шаг.
    
    caps = caps.copy()  #чтобы не писать в старые массивы, а создавать новые, 
    share = share.copy() #вдруг мы решим историю хранить 
    
    create_new_companies(caps,step)
    fund_invest_new_companies(caps, share, cash, step)
    
    progress_old_companies(caps, share, cash, step)
    fund_invest_old_companies(caps, share, cash, step)
    
    return (caps,share,cash)

In [ ]:
columns=["cap_05","cap","cap_95","cash_05","cash","cash_95"]
results = pd.DataFrame(np.zeros((time_length,len(columns)),dtype=float),columns=columns)

for time in range(time_length):
    capitalizations, fund_share, fund_cash = time_step(capitalizations, fund_share, fund_cash, time)
    fund_cap = np.sum(capitalizations * fund_share, axis=1) + fund_cash
    
    a=np.percentile(a=fund_cap,q=(5,50,95))
    results.iloc[time,0:3]=a
    b=np.percentile(a=fund_cash,q=(5,50,95))
    results.iloc[time,3:6]=b
    print(results.iloc[time,:].values.round(-3))
    
results.plot()

[  6.00000000e+09   6.00000000e+09   6.00000000e+09   5.48522700e+09
   5.53543800e+09   5.58466900e+09]
[  8.62258600e+09   9.05896600e+09   9.52816500e+09   7.61566600e+09
   8.05818000e+09   8.52043800e+09]
[  1.47240870e+10   1.57958120e+10   1.69618930e+10   1.31105940e+10
   1.41783830e+10   1.53399580e+10]
[  2.47086780e+10   2.68579410e+10   2.95538970e+10   2.24634550e+10
   2.45554820e+10   2.71535430e+10]
[  3.91311720e+10   4.30921230e+10   4.85661030e+10   3.61321480e+10
   3.99698400e+10   4.53088040e+10]
[  5.88205680e+10   6.53530990e+10   7.61607750e+10   5.49283400e+10
   6.13076510e+10   7.18318330e+10]
